In [5]:
import fridge
import fridge.utilities.outputInterface as OI
import fridge.utilities.h5Interface as h5i
import fridge.utilities.reactorInterface as RI
import os

# Using the h5Interface

This tutorial will run through how to use the `h5Interface` to examine the outputs of an MCNP file.
This is started by creating an instance of the `h5Interface`.
After this we create the initial h5 file via the `create_h5` method.
We note that an `output_name` can be passed as a keyword argument to title the h5 file that will be created.


In [2]:
h5_interface = h5i.h5Interface()
h5_interface.create_h5()

# Add a Reactor

Once an h5_interface has been built, a reactor can be added to it via the `add_reactor` method.
It requires the name of the outpuf file it is looking, along with a path if it is not in the current directory.

Note: `FRIDGe` expects the output files to be in a `.out` format.

In [3]:
for root, dirs, files in os.walk('/Users/ryanstewart/projects/sfr_database'):
    for file in files:
        if '.out' in file:
            try:
                h5_interface.add_reactor(file, path=root)
            except ValueError:
                pass

# H5 Database Structure

The structure of the H5 database can be seen in the figure below.

# Independent Variables

Now that two reactors have been added, let's examine the independent variables for each reactor.
Currently, the independent variables are assocaited with the output file name.
For example, `FC_FS65_H75_23Pu4U_BU` will create the independent variables for fuel smear, height, plutonium content, uranium content, and the fact that this is the burnup. 
In the future, this may be created when FRIDGe builds the input file.

To access different parameters, the `h5file` attribute should be called.
This points to the `h5py` h5 file.
Reading the h5 file involves accessing different layers, similar to accessing a dictionary.
Below we access the independent variables for both cores that were loaded.

We see that for each core we have an associated height, plutonium content, uranium content, and smear.

In [ ]:
reactor_db = {}
iv = 'independent variables'
for rx, core in h5_interface.h5file.items():
    reactor_db[rx] = {}    
    ind = core['independent variables']
    for k,v in core[iv].items():
        reactor_db[rx][k] = v[0]
print(reactor_db)

# Dependent Variables

Now we want to access information that MCNP generated during its run.
To do this, we access the `rx_parameters` of the reactor in question.
Each specific reactor will have a set of `rx_parameters` based on the MCNP output.
For example, if we are curious about the eigenvalue (keff) for the begining and end cycle.
We need to access the `BU` file for each reactor type, and then examine `step_0` and `step_6` to determine the eigenvalue.
Where `step_0` and `step_6` are only signficant because we know that those are the begining and end of cycle burnup steps
Where we now have a set of independent variables that map to our begining of life and end of life eigenvalues.

In [32]:
reactor_db = {}
for rx, core in h5_interface.h5file.items():
    try:
        rx_ = RI.reactorInterface(core)
        print('Reactor Dependent Variables for {} \n Doppler: {} \n Void: {} \n Rx Swing: {} \n Keff EOL: {} \n Peak tp Avg BU {} \n Peak to Avg Power: {}'.format(rx,
        rx_.get_doppler_coefficient(), rx_.get_void_coefficient(), rx_.get_reactivity_swing('step_0', 'step_6'),
        rx_.rx_step_params['step_0']['keff'][0], rx_.get_peak_to_average('step_6', 'burnup'), 
        rx_.get_peak_to_average('step_6', 'power fraction')))
        print('Independent Variables \n Height: {} \n Smear: {} \n Pu Fraction: {}'.format(
            rx_.rx['independent variables']['height'][0],
            rx_.rx['independent variables']['smear'][0],
            rx_.rx['independent variables']['pu_content'][0]))
    except KeyError:
              pass

Reactor Dependent Variables for FS55_H75_23Pu4U10Zr 
 Doppler: -0.5820354163480328 
 Void: -79.36797229215306 
 Rx Swing: 7226.169132950501 
 Keff EOL: 1.1164 
 Peak tp Avg BU ('202', 1.3160853821231182) 
 Peak to Avg Power: ('142', 1.2766293608953385)
Independent Variables 
 Height: 75.0 
 Smear: 55.0 
 Pu Fraction: 0.875
Reactor Dependent Variables for FS60_H80_27Pu0U10Zr 
 Doppler: -0.48686249934863235 
 Void: -56.396218698933566 
 Rx Swing: 5597.205374500647 
 Keff EOL: 1.20861 
 Peak tp Avg BU ('162', 1.3288257163369654) 
 Peak to Avg Power: ('222', 1.295106327235492)
Independent Variables 
 Height: 80.0 
 Smear: 60.0 
 Pu Fraction: 1.0
Reactor Dependent Variables for FS65_H75_11Pu16U10Zr 
 Doppler: -0.4612819658797917 
 Void: -87.2780240707277 
 Rx Swing: 5558.10977824278 
 Keff EOL: 1.10403 
 Peak tp Avg BU ('122', 1.3241303966659024) 
 Peak to Avg Power: ('222', 1.2889400428586717)
Independent Variables 
 Height: 75.0 
 Smear: 65.0 
 Pu Fraction: 0.4
Reactor Dependent Variables

# Assembly Variables

Now we want to examine information on an assembly basis, rather than integral core parameters.
Similar to the `rx_parameters`, we can examine assembly paramters in the `assemblies` directory.


In [43]:
def avg_assem(core, param):
    a_tot = 0
    for num, assem in enumerate(core.values()):
        a_p = assem[param][0]
        a_tot += a_p
    return a_tot/num

def peak_to_average(core, param):
    a_peak = 0
    a_avg = avg_assem(core, param)
    for assem in core.values():
        if assem[param][0] > a_peak:
            a_peak = assem[param][0]
    return a_peak/a_avg
    
reactor_db = {}
for rx, core in h5_interface.h5file.items():
    step = 'step_6'
    try:
        core_step = core['{}_BU'.format(rx)][step]
        core_0 = core['{}_BU'.format(rx)]['step_0']
        assem = core_step['assemblies']
        assem_power = assem['122']['power fraction'][0]
        avg_power = round(avg_assem(assem, 'power fraction') * 300, 3)
        peak_power = round(peak_to_average(assem, 'power fraction') * avg_power, 3)
        assem_bu = assem['122']['burnup'][0]
        avg_bu = round(avg_assem(assem, 'burnup'),3)
        peak_bu = round(peak_to_average(assem, 'burnup'),3)
        keff_0 = core_0['rx_parameters']['keff'][0]
        keff_step = core_step['rx_parameters']['keff'][0]
        rx_swing = round((keff_0 - keff_step)/(keff_0*keff_step) * 1E5, 2)
        print('Assembly Properties for {} \n Assembly Average BU: {} \n Average Power: {} \n Peak Burnup: {} \n Peak Power: {} \n Rx Swing: {}'.format(rx, avg_bu, avg_power, peak_bu, peak_power, rx_swing))
    except KeyError:
        pass


Assembly Properties for FS55_H75_23Pu4U10Zr 
 Assembly Average BU: 44.974 
 Average Power: 3.896 
 Peak Burnup: 1.316 
 Peak Power: 4.974 
 Rx Swing: 7226.17
Assembly Properties for FS60_H80_27Pu0U10Zr 
 Assembly Average BU: 38.53 
 Average Power: 3.896 
 Peak Burnup: 1.329 
 Peak Power: 5.046 
 Rx Swing: 5597.21
Assembly Properties for FS65_H75_11Pu16U10Zr 
 Assembly Average BU: 38.516 
 Average Power: 3.896 
 Peak Burnup: 1.324 
 Peak Power: 5.022 
 Rx Swing: 5558.11
Assembly Properties for FS65_H75_23Pu4U10Zr 
 Assembly Average BU: 38.055 
 Average Power: 3.896 
 Peak Burnup: 1.332 
 Peak Power: 5.052 
 Rx Swing: 5376.02
Assembly Properties for FS65_H75_3Pu24U10Zr 
 Assembly Average BU: 38.815 
 Average Power: 3.896 
 Peak Burnup: 1.32 
 Peak Power: 5.013 
 Rx Swing: 5610.42
Assembly Properties for FS70_H50_27Pu0U10Zr 
 Assembly Average BU: 52.841 
 Average Power: 3.896 
 Peak Burnup: 1.323 
 Peak Power: 4.971 
 Rx Swing: 8446.42
Assembly Properties for FS70_H80_27Pu0U10Zr 
 Assembl

In [18]:
h5_interface.h5file['FS65_H75_23Pu4U10Zr'].keys()#['FS65_H75_23Pu4U10Zr']['step_0']['assemblies'].keys()

<KeysViewHDF5 ['FS65_H75_23Pu4U10Zr_600K', 'FS65_H75_23Pu4U10Zr_BU', 'FS65_H75_23Pu4U10Zr_BU-checkpoint', 'FS65_H75_23Pu4U10Zr_Void', 'independent variables']>

In [ ]:
def avg_assem(core, param):
    a_tot = 0
    for num, assem in enumerate(core.values()):
        a_p = assem[param][0]
        a_tot += a_p
    return a_tot/num

def peak_to_average(core, param):
    a_peak = 0
    a_avg = avg_assem(core, param)
    for assem in core.values():
        if assem[param][0] > a_peak:
            a_peak = assem[param][0]
    return a_peak/a_avg
    
    
reactor_db = {}
for rx, core in h5_interface.h5file.items():
    ind = core['independent variables']
    keff_bol = core['{}_BU'.format(rx)]['step_0']['rx_parameters']['keff'][0]
    keff_eol = core['{}_BU'.format(rx)]['step_6']['rx_parameters']['keff'][0]
    assem_power = core['{}_BU'.format(rx)]['step_6']['assemblies']['122']['power fraction'][0]
    avg_power = round(avg_assem(core['{}_BU'.format(rx)]['step_6']['assemblies'], 'power fraction') * 300, 3)
    peak_power = round(peak_to_average(core['{}_BU'.format(rx)]['step_6']['assemblies'], 'power fraction') * avg_power, 3)
    assem_bu = core['{}_BU'.format(rx)]['step_6']['assemblies']['122']['burnup'][0]
    avg_bu = round(avg_assem(core['{}_BU'.format(rx)]['step_6']['assemblies'], 'burnup'),3)
    peak_bu = round(peak_to_average(core['{}_BU'.format(rx)]['step_6']['assemblies'], 'burnup'),3)
    reactor_db[rx] = {'ind': (ind['height'][0], ind['smear'][0], ind['pu_content'][0], ind['u_content'][0]), 
                      'obj': (keff_bol, keff_eol, assem_power, avg_power, peak_power, assem_bu)}
    print('Reactor Properties for {} \n Average BU: {} \n Average Power: {} \n Peak Burnup: {} \n Peak Power: {}'.format(rx, avg_bu, avg_power, peak_bu, peak_power))


#print(reactor_db)
